In [1]:
import pandas as pd
import numpy as np
import ast
import seaborn
from collections import Counter
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer("english")
from bs4 import BeautifulSoup
import logging, gensim, bz2
import os
import re
from operator import itemgetter

In [2]:
def load(filename):
    file = open(filename,'rb')
    obj = pickle.load(file)
    file.close()
    return obj

def dump(obj,filename):
    filehandler = open(filename,"wb")
    pickle.dump(obj,filehandler)
    filehandler.close()

In [68]:
business_data = pd.read_csv("../data/yelp_academic_dataset_business.csv")

In [71]:
pitt_business_data = business_data[business_data["city"]=="Pittsburgh"]

In [72]:
pitt_business_data.head(n=5)

,neighborhood,business_id,hours,is_open,address,attributes,categories,city,review_count,name,longitude,state,stars,latitude,postal_code,type
38,West View,P3LisOj7DktgGa7C5FYpnA,"['Monday 9:30-9:0', 'Tuesday 9:30-9:0', 'Wedne...",1,451 Rochester Rd,['BusinessAcceptsCreditCards: True'],"['Plumbing', 'Home Services']",Pittsburgh,8,Benjamin Franklin Plumbing,-80.049887,PA,4.0,40.534163,15237,business
44,North Side,93otbGHE0s0m-lU1osvg9w,NaN,1,337 N Shore Dr,"['Alcohol: full_bar', ""Ambience: {'romantic': ...","['American (New)', 'Pubs', 'Nightlife', 'Bars'...",Pittsburgh,102,Rivertowne,-80.010880,PA,3.0,40.445986,15212,business
56,Greentree,csdZolWIWvkIHIqsCmV6sg,"['Monday 10:0-18:0', 'Tuesday 10:0-18:0', 'Wed...",1,325 Elizabeth Dr,"['BusinessAcceptsCreditCards: True', 'ByAppoin...","['Hotels & Travel', 'Home Services', 'Hotels',...",Pittsburgh,5,City Vista,-80.037872,PA,2.0,40.426386,15220,business
81,Bloomfield,h7ZW6c1Vi9V6_-gd2Ch_4g,"['Monday 10:0-21:0', 'Tuesday 10:0-21:0', 'Wed...",1,4815 Centre Ave,"['BusinessAcceptsCreditCards: True', ""Business...","['Shopping', 'Mobile Phones']",Pittsburgh,5,Verizon Wireless Premium Retailer,-79.946995,PA,2.0,40.453725,15213,business
107,Strip District,HWrbZS1mxVRj2Y2VwMmDMg,NaN,0,2627 Penn Ave,"['BikeParking: True', 'BusinessAcceptsCreditCa...","['Food', 'Bakeries']",Pittsburgh,3,Dulcinea Bakeshop,-79.977728,PA,4.5,40.455351,15222,business


In [67]:
pitt_review.head(n=5)

,funny,user_id,review_id,text,business_id,stars_x,date,useful,type_x,cool,...,review_count,name,longitude,state,stars_y,latitude,postal_code,type_y,Restaurant,clean_text
1032,1,-UPUYET3Pwm99zq-uHl7gg,OPBHCs-lcEVWbEZY7rKNUg,I would give this a 3 1/2. Typical bar food s...,0Wy4gw8krao9nGq-sHWFSQ,3,2013-12-01,1,review,1,...,41,Brewstone,-79.809634,PA,2.5,40.429825,15235,business,1,"[would, give, typic, bar, food, select, food, ..."
1033,0,WLrFY6_Z32lb_ifcIvRvkw,SjgayD75MU7yTWEs_cGbwg,It was hit or miss for us. \n\nWe got burgers ...,0Wy4gw8krao9nGq-sHWFSQ,3,2013-05-06,0,review,0,...,41,Brewstone,-79.809634,PA,2.5,40.429825,15235,business,1,"[hit, miss, u, got, burger, first, bread, look..."
1034,0,pOQduF1Wf8fQeXlWEmNH2w,0cBOrzOUqdm-eZHQJDbKAA,My girlfriend and I showed up at 7:30 on a Fri...,0Wy4gw8krao9nGq-sHWFSQ,1,2013-06-04,0,review,0,...,41,Brewstone,-79.809634,PA,2.5,40.429825,15235,business,1,"[girlfriend, show, 30, friday, night, complet,..."
1035,0,UyhtAbEnjTtp-v0CQneh6Q,WY3Bl7Huzv6Bzg-gMmnyMg,Absolutely terrible! Just got up and left...se...,0Wy4gw8krao9nGq-sHWFSQ,1,2014-09-08,0,review,0,...,41,Brewstone,-79.809634,PA,2.5,40.429825,15235,business,1,"[absolut, terribl, got, left, seat, 15, minut,..."
1036,0,3QHmbVzpWPM2PKm4kAI01Q,vPcoXzJbA9U77RYkPwxXbw,"I had heard some bad reviews about Brewstone, ...",0Wy4gw8krao9nGq-sHWFSQ,4,2012-08-12,2,review,0,...,41,Brewstone,-79.809634,PA,2.5,40.429825,15235,business,1,"[heard, bad, review, brewston, especi, servic,..."


In [8]:
review_with_topics = load("../Pickled_files/reviews_with_topics.pkl")

In [13]:
review_with_topics.head(n=10)

,index,user_id,review_id,text,name,attributes,categories,stars_x,stars_y,Top_ID,Top_name,Top_level2,Score_top,All_ID,All_name,All_level2,All_score,Top_Topic_Words
0,1032,-UPUYET3Pwm99zq-uHl7gg,OPBHCs-lcEVWbEZY7rKNUg,I would give this a 3 1/2. Typical bar food s...,Brewstone,"['Alcohol: full_bar', ""Ambience: {'romantic': ...","[American (New), Restaurants]",3,2.5,8,service,service_food_quantity_price,0.432233,"[8, 11, 46, 39, 29, 30, 42, 2, 22]","[service, Bar, Bar, food_quality, food_quality...","[service_food_quantity_price, alchol_beer_bar,...","[(8, 0.432232734338), (11, 0.177604856512), (4...","[good, food, price, place, pretti, servic, rea..."
1,1033,WLrFY6_Z32lb_ifcIvRvkw,SjgayD75MU7yTWEs_cGbwg,It was hit or miss for us. \n\nWe got burgers ...,Brewstone,"['Alcohol: full_bar', ""Ambience: {'romantic': ...","[American (New), Restaurants]",3,2.5,8,service,service_food_quantity_price,0.260327,"[8, 28, 30, 46, 35, 40, 6, 34, 11, 0]","[service, cusine, service, Bar, ambience, Need...","[service_food_quantity_price, burger_friday, t...","[(8, 0.26032685149), (28, 0.158365186441), (30...","[good, food, price, place, pretti, servic, rea..."
2,1034,pOQduF1Wf8fQeXlWEmNH2w,0cBOrzOUqdm-eZHQJDbKAA,My girlfriend and I showed up at 7:30 on a Fri...,Brewstone,"['Alcohol: full_bar', ""Ambience: {'romantic': ...","[American (New), Restaurants]",1,2.5,34,service,waiting_time_seat,0.444599,"[34, 2, 46, 11, 18, 25]","[service, service, Bar, Bar, NeedsLabelling, a...","[waiting_time_seat, service, bar, alchol_beer_...","[(34, 0.444599435235), (2, 0.312317425266), (4...","[wait, minut, u, tabl, order, seat, time, get,..."
3,1035,UyhtAbEnjTtp-v0CQneh6Q,WY3Bl7Huzv6Bzg-gMmnyMg,Absolutely terrible! Just got up and left...se...,Brewstone,"['Alcohol: full_bar', ""Ambience: {'romantic': ...","[American (New), Restaurants]",1,2.5,34,service,waiting_time_seat,0.705142,"[34, 2]","[service, service]","[waiting_time_seat, service]","[(34, 0.70514157441), (2, 0.226286997019)]","[wait, minut, u, tabl, order, seat, time, get,..."
4,1036,3QHmbVzpWPM2PKm4kAI01Q,vPcoXzJbA9U77RYkPwxXbw,"I had heard some bad reviews about Brewstone, ...",Brewstone,"['Alcohol: full_bar', ""Ambience: {'romantic': ...","[American (New), Restaurants]",4,2.5,46,Bar,bar,0.189172,"[46, 8, 30, 11, 2, 44, 23, 47, 6, 25, 35, 28, ...","[Bar, service, service, Bar, service, NeedsLab...","[bar, service_food_quantity_price, time, alcho...","[(46, 0.189171509388), (8, 0.171203052936), (3...","[bar, drink, night, bartend, game, music, frie..."
5,1037,Pm5tq0IzkfX4iINnI8ZGTw,cmu62LdCBwqKiDDHciNXPg,Nice design and sporty atmosphere surround you...,Brewstone,"['Alcohol: full_bar', ""Ambience: {'romantic': ...","[American (New), Restaurants]",3,2.5,8,service,service_food_quantity_price,0.307530,"[8, 29, 21, 46, 11, 5, 0, 24]","[service, food_quality, cusine, Bar, Bar, Need...","[service_food_quantity_price, menu, wings_BBQ_...","[(8, 0.307530380229), (29, 0.278536930681), (2...","[good, food, price, place, pretti, servic, rea..."
6,1038,_VTEyUzzH92X3w-IpGaXVA,raC3BDXbIDuE3HDcqjEMnw,I got the feeling that Brewstone isn't exactly...,Brewstone,"['Alcohol: full_bar', ""Ambience: {'romantic': ...","[American (New), Restaurants]",3,2.5,29,food_quality,menu,0.285777,"[29, 46, 8, 6, 39, 30, 11, 23, 28, 13, 25, 14,...","[food_quality, Bar, service, food_quality, foo...","[menu, bar, service_food_quantity_price, food_...","[(29, 0.285776575906), (46, 0.175881860623), (...","[restaur, menu, tabl, meal, dine, u, one, dinn..."
7,1039,y30lxxDg0AlzdbeWLzHedw,EJS_zLsSBw4lEVsAqWo7LQ,"This place is in transition from ""Brewstone Be...",Brewstone,"['Alcohol: full_bar', ""Ambience: {'romantic': ...","[American (New), Restaurants]",3,2.5,46,Bar,bar,0.217104,"[46, 8, 35, 47, 11, 2, 23, 29, 21, 40, 33, 14,...","[Bar, service, ambience, service, Bar, service...","[bar, service_food_quantity_price, ambience, s...","[(46, 0.217103590217), (8, 0.168434175005), (3...","[bar, drink, night, bartend, game, music, frie..."
8,1040,n

In [77]:
simp_rate_top = review_with_topics.groupby(["name","Top_name"])["stars_x"].mean()
simp_rate_top = simp_rate_top.unstack()
simp_rate_top = pd.merge(simp_rate_top,pitt_business_data[["name","stars"]],left_index=True,right_on="name")

In [85]:
cols = simp_rate_top.columns.tolist()
cols = cols[-2:] + cols[:-2]
simp_rate_top = simp_rate_top[cols]

In [148]:
simp_rate_top.to_csv("../Pickled_files/simp_rate_top.csv")
dump(simp_rate_top,"../Pickled_files/simp_rate_top.pkl")

## Trying different method

In [96]:
list_dict = map(lambda x: Counter(x),review_with_topics["All_name"])

In [128]:
topic_cat = pd.DataFrame(list_dict)

In [150]:
simp_rate_all = pd.concat([review_with_topics[["review_id","name","stars_x"]],topic_cat],axis=1)
simp_rate_all = pd.melt(simp_rate_all,id_vars=["review_id","name","stars_x"],value_vars=topic_cat.columns.tolist())
simp_rate_all = simp_rate_all[simp_rate_all["value"].notnull()]
simp_rate_all = simp_rate_all.groupby(["name","variable"])["stars_x"].mean()
simp_rate_all= simp_rate_all.unstack()
simp_rate_all = pd.merge(simp_rate_all,pitt_business_data[["name","stars"]],left_index=True,right_on="name")

In [151]:
cols = simp_rate_all.columns.tolist()
cols = cols[-2:] + cols[:-2]
simp_rate_all = simp_rate_all[cols]

In [152]:
simp_rate_all.to_csv("../Pickled_files/simp_rate_all.csv")
dump(simp_rate_all,"../Pickled_files/simp_rate_all.pkl")